All imports ??

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver

Initialize MemorySaver Object ??

In [2]:
memory = MemorySaver()

Defination of the State ??

In [3]:
class State(TypedDict):
    value: str

Definition of the first node in the graph ??

In [4]:
def node_a(state: State):
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "value": state["value"] + "a"
        }
    )

Definition of the second node in the graph with interrupt ??

In [5]:
def node_b(state: State):
    print("Node B")
    human_response = interrupt("Do you want to go to C or D? Type (C/D): ")
    
    print("Human Review Values: ", human_response)
    
    if human_response == "C":
        return Command(
            goto="node_c",
            update={
                "value": state["value"] + "b"
            }
        )
    elif human_response == "D":
        return Command(
            goto="node_d",
            update={
                "value": state["value"] + "b"
            }
        )

Definition of the third node in the graph ??

In [6]:
def node_c(state: State):
    print("Node C")
    return Command(
        goto=END,
        update={
            "value": state["value"] + "c"
        }
    )

Definition of the forth node in the graph ??

In [7]:
def node_d(state: State):
    print("Node D")
    return Command(
        goto=END,
        update={
            "value": state["value"] + "d"
        }
    )

Start Building the graph ??

In [8]:
graph_builder = StateGraph(State)

graph_builder.add_node("node_a", node_a)
graph_builder.add_node("node_b", node_b)
graph_builder.add_node("node_c", node_c)
graph_builder.add_node("node_d", node_d)

graph_builder.set_entry_point("node_a")

graph = graph_builder.compile(checkpointer=memory)

Testing The Graph ??

In [12]:
config = {
    "configurable": {"thread_id": "1"}
}

initialState = {
    "value": ""
}

first_response = graph.invoke(initialState, config, stream_mode="updates")

first_response

Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type (C/D): ', id='6d684526e055577d80bd852c68d50ea6'),)}]

In [13]:
graph.get_state(config).next

('node_b',)

In [14]:
second_response = graph.invoke(Command(resume="D"), config=config, stream_mode="updates")

second_response

Node B
Human Review Values:  D
Node D


[{'node_b': {'value': 'ab'}}, {'node_d': {'value': 'abd'}}]

In [15]:
graph.get_state(config).next

()